* one convolution notebook

In [3]:
# tensorflow on 8889

In [5]:
import os
os.chdir("/notebooks/src")

In [1]:
pwd

u'/notebooks/src'

In [4]:
! ls /home

In [7]:
!pip install keras cython h5py

    100% |################################| 276kB 2.2MB/s ta 0:00:01
    100% |################################| 2.9MB 450kB/s eta 0:00:01
    100% |################################| 256kB 5.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/2c/f7/79/13f3a12cd723892437c0cfbde1230ab4d82947ff7b3839a4fc
Successfully built pyyaml


In [126]:
import keras
from keras.layers import Input,Convolution2D, Dense, Activation, Flatten, merge, Conv2D
from keras.layers import MaxPooling2D, Dropout, LocallyConnected2D, MaxPool2D, MaxPool1D
from keras.models import Model, load_model,  Sequential
from keras.optimizers import SGD
import numpy as np
import pandas as pd
#from lib.data_split import load_train_data, load_test_data

### preprocessing / splitting

In [189]:
train_df = pd.read_csv("./data/train.csv")

In [190]:
test_df = pd.read_csv("./data/test.csv")

In [191]:
# function to organize the data
def encode(train_df, test_df):
    le = LabelEncoder().fit(train_df.species)
    labels = le.transform(train_df.species)   #encode species strings
    classes = list(le.classes_)
    test_ids = test_df.id
    
    
    train_df = train_df.drop(["species" , "id"] , axis = 1)
    test_df =test_df.drop(["id"], axis =1)
    
    return train_df, labels, test_df, test_ids, classes

In [192]:
train_df, labels, test_df, test_ids, classes = encode(train_df, test_df)
train_df.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,0.007812,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,0.044922,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,0.017578,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,0.005859,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [194]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(train_df)
scaled_test_data = scaler.transform(test_df)

In [196]:
X = train_df.values
y = keras.utils.to_categorical(labels)

In [198]:
sss = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2, random_state= 8)
sss.get_n_splits(X, y)

1

In [199]:
for train_index, test_index in sss.split(X,y):
    X_train, X_test = X[train_index],X[test_index]
    y_train, y_test = y[train_index],y[test_index]

In [200]:
X_train.shape

(792, 192)

In [201]:
y_train.shape

(792, 99)

### preprocessing the data(functions)

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
'''
This module takes a file path and loades pre_extracted features 
from leaf kaggle competiton. The input is the file path for
train or test files. 
'''



def load_train(filepath):

    #loading
    train_df = pd.read_csv(filepath)
    id = train_df.pop('id')
    target = train_df.pop('species')
    
    #scaling
    target = LabelEncoder().fit(target).transform(target)
    features = StandardScaler().fit(train_df).transform(train_df)
    
    return id,features,target
    

def load_test(filepath):
    
    #loading
    test_df = pd.read_csv(filepath)
    id = test_df.pop('id')
    
    #scaling
    features = StandardScaler().fit(test_df).transform(test_df)
    
    return id, features
    

In [38]:
#train_df.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,0.007812,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,0.044922,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,0.017578,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,0.005859,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [39]:
#test_df.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,0.023438,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
1,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,0.023438,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
2,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,0.005859,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
3,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,0.001953,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
4,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,0.000000,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


* loading images

In [10]:
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import img_to_array, load_img

'''
This module loads all the training/test images, resizes them and 
makes them ready for kaggle.
'''


def rebuild_image(img, dim=64):
    
    # find largest axis
    maxsize = max((0,1), key=lambda i: img.size[i])
    
    #scale both axis
    scale = dim/float(img.size[maxsize])
    
    return img.resize((int(img.size[0] * scale), int(img.size[1]*scale)))
    

    
def load_image(ids,dim=64):

    filepath = './data/images/'
    
    IM = np.empty((len(ids),dim,dim,1))
    
    for i, ids in enumerate(ids):
    
        images = rebuild_image(load_img(filepath + str(ids) + '.jpg',
grayscale=True),dim = dim)
        
        images = img_to_array(images)
        x = images.shape[0]
        y = images.shape[1]
        
        #centering
        l_bound_0 = int((dim - x) / 2)
        u_bound_0 = l_bound_0 + x
        l_bound_1 = int((dim - y) / 2)
        u_bound_1 = l_bound_1 + y
        
        IM[i,l_bound_0:u_bound_0,l_bound_1:u_bound_1,0:1] = images
        
        #scale it to grey
    return np.around(IM/ 255.0) 

### splitting the data with StratifiedShuffleSplit

In [15]:

from sklearn.model_selection import StratifiedShuffleSplit

random_state = 8
split = 0.8





def load_train_data(filepath, split = split, random_state = random_state):

    id, features_train, target_train = load_train(filepath)
    #image_train = load_image(id)
    
    sss = StratifiedShuffleSplit(n_splits = 1, train_size=split,random_state=random_state)
    train_i,val_i = next(sss.split(features_train,target_train))
    #,val_img   #,image_train[val_i]
    features_val,val_target = features_train[val_i], target_train[val_i]
    #,train_img  #,image_train[train_i]
    features_train,train_target = features_train[train_i],target_train[train_i]
    
    #,train_img   #,val_img
    return (features_train,train_target),(features_val,val_target)
    

def load_test_data(filepath):

    id, test_features = load_test(filepath)
    #test_images = load_image(id)
    
    return (id,test_features)#,test_images)
    
    

In [16]:
#X_train, y_train
train,validation = load_train_data("./data/train.csv")

In [17]:
test = load_test_data("./data/test.csv")

In [46]:
type(test)

tuple

In [64]:
train_labels = train[1][:]
valid_labels = validation[1][:]

In [63]:
train_labels.shape

(792, 99)

In [66]:
valid_labels.shape

(99,)

In [24]:
train[1][:].shape

(792,)

In [25]:
validation[1][:].shape

(99,)

### reformat function

In [85]:
def reformat(labels):
    labels = (np.arange(99) == labels[:,None]).astype(np.float32)
    return labels
train_labels = reformat(train_labels)
valid_labels = reformat(valid_labels)

# right after here!!!

In [28]:
#input_layer = Input(shape=(64,64,1))

 ### The Convolutional Neural Network(CNN)

In [131]:
#the uotput layer the last layer should be the number of your classes.(here it's 99)
# the input_shape should be like 
dnn_clf = Sequential([
#     Conv2D(8, kernel_size=(3, 3),
#                  border_mode= "same",
#                  W_regularizer= "l1",
#                  activation='relu',
#                  input_shape=(192,1)),
    Dense(100, input_shape=(192, 1), activation='relu'),
#     Conv2D(16, (3, 3), activation='relu'),
#     Dropout(.25),
    MaxPool1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(.25),
    Dense(99, activation='softmax')
])


In [132]:
dnn_clf.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 192, 100)          200       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 96, 100)           0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 9600)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 100)               960100    
_________________________________________________________________
dropout_17 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 99)                9999      
Total params: 970,299
Trainable params: 970,299
Non-trainable params: 0
_________________________________________________________________


### compile on (optimizer = Adam)

In [133]:
cnn.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

In [134]:
#cnn.fit(X_train, y_train, validation_split=.3, epochs=3)

In [69]:
train[1][:].shape

(792,)

In [72]:
train_labels.shape

(792, 99)

In [73]:
valid_labels.shape

(99, 99)

In [202]:
X2_train = X_train.reshape(-1, 192, 1)

In [203]:
X_train.shape

(792, 192)

In [204]:
X2_train.shape

(792, 192, 1)

In [ ]:
# adding layers or depth might not help the scores any more.
# but you can add random_forest and other classifiers prediction to the end of your data set that might help you
# get a better score.


In [206]:
cnn.fit(X2_train,y_train, validation_split=.3, epochs=100)
                    

Train on 554 samples, validate on 238 samples
Epoch 1/100
554/554 [==============================] - 0s - loss: 4.4145 - acc: 0.1155 - val_loss: 4.4570 - val_acc: 0.0798
Epoch 2/100
554/554 [==============================] - 0s - loss: 4.2845 - acc: 0.1625 - val_loss: 4.3601 - val_acc: 0.1092
Epoch 3/100
554/554 [==============================] - 0s - loss: 4.1209 - acc: 0.1805 - val_loss: 4.2028 - val_acc: 0.1429
Epoch 4/100
554/554 [==============================] - 0s - loss: 3.8833 - acc: 0.2166 - val_loss: 4.0217 - val_acc: 0.1639
Epoch 5/100
554/554 [==============================] - 0s - loss: 3.6192 - acc: 0.2563 - val_loss: 3.7954 - val_acc: 0.2101
Epoch 6/100
554/554 [==============================] - 0s - loss: 3.3522 - acc: 0.2780 - val_loss: 3.5314 - val_acc: 0.2647
Epoch 7/100
554/554 [==============================] - 0s - loss: 3.0922 - acc: 0.3484 - val_loss: 3.2654 - val_acc: 0.2983
Epoch 8/100
554/554 [==============================] - 0s - loss: 2.7952 - acc: 0.3700

554/554 [==============================] - 0s - loss: 0.1856 - acc: 0.9531 - val_loss: 0.6089 - val_acc: 0.8445
Epoch 67/100
554/554 [==============================] - 0s - loss: 0.1671 - acc: 0.9693 - val_loss: 0.6001 - val_acc: 0.8361
Epoch 68/100
554/554 [==============================] - 0s - loss: 0.1631 - acc: 0.9585 - val_loss: 0.5940 - val_acc: 0.8487
Epoch 69/100
554/554 [==============================] - 0s - loss: 0.1662 - acc: 0.9657 - val_loss: 0.6107 - val_acc: 0.8487
Epoch 70/100
554/554 [==============================] - 0s - loss: 0.1680 - acc: 0.9549 - val_loss: 0.6282 - val_acc: 0.8361
Epoch 71/100
554/554 [==============================] - 0s - loss: 0.1616 - acc: 0.9585 - val_loss: 0.6192 - val_acc: 0.8403
Epoch 72/100
554/554 [==============================] - 0s - loss: 0.1563 - acc: 0.9585 - val_loss: 0.5852 - val_acc: 0.8529
Epoch 73/100
554/554 [==============================] - 0s - loss: 0.1580 - acc: 0.9675 - val_loss: 0.5729 - val_acc: 0.8571
Epoch 74/100


In [208]:
X2_test = X_test.reshape(-1, 192, 1)

In [211]:
y_preds = cnn.predict_classes(X2_test)

 32/198 [===>..........................] - ETA: 0s

In [220]:
y_preds

array([36, 44, 32, 36, 61, 62, 17, 55, 58, 75,  2, 78, 84, 48, 28, 93, 59,
       74,  9, 92, 16, 25, 65, 11, 65, 39, 98, 64,  7, 66, 62, 93, 96, 14,
       47, 94, 35,  5, 22, 72, 51, 49, 74, 48, 89, 98, 21,  2, 56, 64, 82,
       76, 56, 88, 57, 37, 95, 18, 39, 26, 13, 85, 83, 46,  8, 42,  4, 30,
       20, 64, 86, 11, 63, 16, 95, 50,  4, 45, 48, 66, 60, 43, 25, 15, 47,
       37, 50, 86, 70, 45, 97, 17, 96, 45, 33, 68, 30, 10, 94,  2, 67, 14,
       42, 27, 89, 52, 73, 41, 54, 22, 92, 77, 87, 34, 79, 85,  3, 41, 81,
       49, 21, 91, 82, 45, 41, 61, 84, 51, 88, 86, 29, 78,  9, 83,  1,  0,
       54, 73, 38, 69, 35, 71, 72, 44, 19, 76, 57, 90, 43, 43, 23, 29, 15,
       80,  6, 19, 43, 23, 87, 68, 47, 43,  7, 75,  0, 50, 97, 60, 91, 33,
       79, 56, 58, 32, 37, 67, 53,  6, 60, 40, 40, 56, 80, 63, 28, 27, 48,
       71, 20, 90, 70, 34, 13, 46, 77,  8, 59,  3])

In [215]:
y_test = np.argmax(y_test, axis=1)

In [222]:
y_test = keras.utils.to_categorical(y_test)

In [224]:
cnn.evaluate(X2_test, y_test)

 32/198 [===>..........................] - ETA: 0s

[0.42065755029519397, 0.89898989898989901]

In [218]:
#turn your y_test back to argmax for confusion matrix results.

from sklearn.metrics import confusion_matrix

In [219]:
confusion_matrix(y_test, y_preds)

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 2, 0],
       [0, 0, 0, ..., 0, 0, 2]])

### compile on(optimizer= Adadelta)

In [89]:
#cnn.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])

## buiding the pipe

In [31]:
#kernel_size: specifying the width and height of the 2D convolution window.
        

pipe_line = Convolution2D(8,5,5, border_mode = "same", W_regularizer='l1')(input_layer)
pipe_line =(Activation("relu"))(pipe_line)
pipe_line = (MaxPooling2D(pool_size=(2,2), strides = (2,2)))(pipe_line)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (5, 5), padding="same", kernel_regularizer="l1")`
  after removing the cwd from sys.path.


In [ ]:
#Conv2D()

## flatten the pipe

In [32]:
#flatten
pipe_line = Flatten()(pipe_line)


## fully connected layer (using dense class)

In [33]:
# fully connected layer
pipe_line = Dense(100, activation="relu")(pipe_line)
pipe_line = Dropout(.5)(pipe_line)


## output layer

In [34]:
#output layer
output_layer = Dense(99, activation="softmax")(pipe_line)

In [35]:
#lr ==> learning rate
sgd = SGD(lr= 0.008, momentum= 0.9)

In [38]:
pipe_line.compile(optimizer = "Adam", loss = "categorical_crossentropy",
             metrics = ["accuracy"]) ## Log_Loss for metrics???

AttributeError: 'Tensor' object has no attribute 'compile'

In [36]:
model_2layer = model.fit(train[1][:], train_labels,
                   validation_data = (validation[1][:],
                                     valid_labels),  nb_epoch = 1)

NameError: name 'model' is not defined